# Data Handling Workflows

### Importing Libraries and Configuration

In [3]:
import numpy as np
import random
import time

import parsl
import os
from parsl.app.app import python_app, bash_app
from parsl.providers import LocalProvider
from parsl.channels import LocalChannel

from parsl.config import Config
from parsl.executors import HighThroughputExecutor

config = Config(
    executors=[
        HighThroughputExecutor(
            label="htex_local",
            cores_per_worker=1,
            provider=LocalProvider(
                channel=LocalChannel(),
                init_blocks=1,
                max_blocks=1,
            ),
        )
    ],
)

parsl.load(config)

### Map Reduce

A map reduce is a technique to execute multiple parallel jobs on a dataset to reduce the size of the dataset before executing a final function to get the result. A Map reduce is a more complicated version of synchronisation.

Let's consider a simple example where we are given multiple lists and we want to select the lists with the highest standard deviation.

![](./images/map_reduce.png)

In [4]:
# A python app to compute the standard deviation of the inputs
@python_app
def standard_deviation(inputs=[]):
    import numpy as np
    return np.std(inputs)

def standard_deviation_non_p(inputs=[]):
    import numpy as np
    return np.std(inputs)

In [5]:
# A function to construct data that is a list of lists, each each list having 100 random numbers

def make_data():
    lists = []
    
    for _ in range(100):
        new_list = []
        
        for __ in range(100):
            new_list.append(random.random()*100)
            
        lists.append(new_list)
    return lists

our_data = make_data()

### Parallel Execution

In [17]:
# Computing the standard deviations for each list

start1 = time.time()

standard_deviations = []

for i in our_data:
    standard_deviations.append(standard_deviation(inputs=i))

# Finding the maximum standard deviation
standard_deviations = [i.result() for i in standard_deviations]
print('Maximum Standard Deviation: ', max(standard_deviations))

# Finding the list with the maximum standard deviation
maximum = max(standard_deviations)
print('Target List Number:', standard_deviations.index(maximum))

end1 = time.time()

Maximum Standard Deviation:  32.32080281855521
Target List Number: 76


### Non-Parallel Execution

In [18]:
start2 = time.time()

standard_deviations = []

for i in our_data:
    standard_deviations.append(standard_deviation_non_p(inputs=i))

# Finding the maximum standard deviation
print('Maximum Standard Deviation: ', max(standard_deviations))

# Finding the list with the maximum standard deviation
maximum = max(standard_deviations)
print('Target List Number:', standard_deviations.index(maximum))

end2 = time.time()

Maximum Standard Deviation:  32.32080281855521
Target List Number: 76


In [19]:
print('Time taken with Parsl: {}'.format(end1-start1))
print('Time taken without Parsl: {}'.format(end2-start2))

Time taken with Parsl: 0.5836498737335205
Time taken without Parsl: 0.010065078735351562


# Hashing

We'll be using a simple hash function to store elements in our database. We'll evaluate the hash values in parallel and then store the items in those locations.

In [12]:
database = [0 for i in range(1000)]  # An empty database

In [20]:
@python_app 
def hash_function(element):
    import hashlib
    number = int(hashlib.md5(element).hexdigest()[:8], 16)%1000   # Creating a hash index
    return number
    
def hash_function_non_p(element):
    import hashlib
    number = int(hashlib.md5(element).hexdigest()[:8], 16)%1000   # Creating a hash index
    return number

### Parallel Execution

In [21]:
database2 = database.copy()

import random

elements = []

for i in range(100):
    element = '' 
    for _ in range(5):
        element += random.choice('abcdefghijklmopqrstuvwxyz')
    element = element.encode() # Making a 5 letter element
    elements.append(element) # collecting 100 such elements

start1 = time.time()   
hashes = []
for i in elements: # Updating the database for all the elements
    hashes.append(hash_function(i))

hashes = [i.result() for i in hashes]

for i in range(len(elements)):
    database[hashes[i]] = elements[i]

end1 = time.time()

### Non-Parallel Execution

In [22]:
start2 = time.time()
    
hashes = []
for i in elements: # Updating the database for all the elements
    hashes.append(hash_function_non_p(i))

for i in range(len(elements)):
    database2[hashes[i]] = elements[i]

end2 = time.time()

In [24]:
print('Time taken with Parsl: {}'.format(end1-start1))
print('Time taken without Parsl: {}'.format(end2-start2))

Time taken with Parsl: 0.5361638069152832
Time taken without Parsl: 0.0007882118225097656


Note that this still doesn't solve the problem of overlap of elements. Chaining is the alternative here but in order to implement chaining, we have to evaluate the results which breaks the parallel thread.

## Montage Mosaic

### First Part

In [ ]:
import os
cwd = os.getcwd()
from parsl.data_provider.files import File

In [ ]:
import montage_wrapper as montage

In [ ]:
!tar xvf Kimages.tar

In [ ]:
montage.mImgtbl(os.path.join(cwd,'Kimages/'),
                File(os.path.join(cwd,'Kimages.tbl')))

In [ ]:
montage.mMakeHdr(File(os.path.join(cwd,'Kimages.tbl')),
                 File(os.path.join(cwd,'Ktemplate.hdr')))

In [ ]:
os.mkdir(os.path.join(cwd,'Kprojdir/'))

In [ ]:
montage.mProjExec(File(os.path.join(cwd,'Kimages.tbl')),
                  File(os.path.join(cwd,'Ktemplate.hdr')),
                  os.path.join(cwd,'Kprojdir/'),
                  File(os.path.join(cwd,'stats.tbl')))

In [ ]:
montage.mImgtbl(os.path.join(cwd,'Kprojdir/'),
                File(os.path.join(cwd,'images.tbl')))

In [ ]:
montage.mAdd( File(os.path.join(cwd,'images.tbl')), 
              File(os.path.join(cwd,'Ktemplate.hdr')), 
              File(os.path.join(cwd,'m17_uncorrected.fits')))

In [ ]:
!mViewer -ct 1 -gray m17_uncorrected.fits -1s max gaussian-log -out m17_uncorrected.png

![](./images/m17_uncorrected.png)

### Second Part

In [ ]:
montage.mOverlaps(File(os.path.join(cwd,'images.tbl')),
                  File(os.path.join(cwd,'diffs.tbl')))

In [ ]:
os.mkdir(os.path.join(cwd,'diffdir/'))

In [ ]:
montage.mDiffExec(File(os.path.join(cwd,'diffs.tbl')), 
                  File(os.path.join(cwd,'Ktemplate.hdr')), 
                  os.path.join(cwd,'diffdir/'),
                  proj_dir=os.path.join(cwd,'Kprojdir/'))

In [ ]:
montage.mFitExec(File(os.path.join(cwd,'diffs.tbl')), 
                 File(os.path.join(cwd,'fits.tbl')), 
                 os.path.join(cwd,'diffdir/'))

In [ ]:
montage.mBgModel(File(os.path.join(cwd,'images.tbl')), 
                 File(os.path.join(cwd,'fits.tbl')), 
                 File(os.path.join(cwd,'corrections.tbl')))

In [ ]:
os.mkdir(os.path.join(cwd,'corrdir'))

In [ ]:
montage.mBgExec(File(os.path.join(cwd,'images.tbl')), 
                File(os.path.join(cwd,'corrections.tbl')), 
                os.path.join(cwd,'corrdir'), 
                proj_dir=os.path.join(cwd,'Kprojdir'))

In [ ]:
montage.mAdd(File(os.path.join(cwd,'images.tbl')), 
             File(os.path.join(cwd,'Ktemplate.hdr')), 
             File(os.path.join(cwd,'m17.fits')))

In [ ]:
!mViewer -ct 1 -gray m17.fits -1s max gaussian-log -out m17.png

![](./images/m17.png)